## LLMs and Chat Models

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

# a = llm.predict("How many planets are there?") // text-davinci-003 2024.01.04 out of service -> 실행 안 됨
b = chat.predict("How many planets are there?") # In service gpt-3.5-turbo

b

'As of August 2021, there are eight recognized planets in our solar system. They are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, it is important to note that in 2006, the International Astronomical Union (IAU) redefined the definition of a planet, leading to the reclassification of Pluto as a "dwarf planet."'

## Predict Messages

In [35]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1 # temperature가 높을수록 창의력이 떨어짐, 답변의 활용도가 낮음
)

In [2]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content = "Ciao, mi chiam Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)


AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

## prompt

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

## chatPromptTemplate : message로부터 template을 만듬
## PromptTemplate : String을 이용해서 template을 만듬
## template에는 검증계층(validation layer)이 있음 -> 사용자가 볼 수 없는 곳에서 template이 질문에 대한 변수를 정의함.

template = PromptTemplate.from_template("what is the distance between {country_a} and {country_b}?")

prompt = template.format(country_a = 'Mexico', country_b="Thailand")
## template.format은 질문을 정리해서 보여줌


print(type(prompt))

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)



<class 'str'>


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

## FewShoutPromptTemplate


In [38]:
# TODO: langchain modules study

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate # 질문을 받고 답변을 할 수 있는 PROMPT를 제작하는 모듈
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate # AI가 답변하는 형식을 학습시킬 수 있는 모듈 -> 대화 방식을 데이터 베이스에 저장하여 가지고 오면 편함
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

t = PromptTemplate(
    template="what is the capital of {country}",
    input_variables=['country'],
)

t.format(country="France")

'what is the capital of France'

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples = [
{
    "question": "what do you know about Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "what do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "what do you kow about France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]
# chat.predict("what do you know about France?")

example_prompt = PromptTemplate.from_template("Human: {question}\nAI{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,
    suffix="Human: What do you know about{country}?",
    input_variables=['country']
)
                                              

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country" : "Turkey"
})


AI
    I know this:
    Capital: Ankara
    Language: Turkish
    Food: Kebab and Baklava
    Currency: Turkish Lira

AIMessageChunk(content='AI\n    I know this:\n    Capital: Ankara\n    Language: Turkish\n    Food: Kebab and Baklava\n    Currency: Turkish Lira')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    model = "gpt-3.5-turbo",
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
{
    "question": "Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?")
    ]
)


final_prompt.format(country = "Japan")

chain = final_prompt | chat

response = chain.invoke(
    {"country" : "Turkey"}
)

print(response)

## 결제????


## LengthBasedExampleSelector

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
# 예제들을 형식화함, 예제의 양이 얼마나 되는지 확인할 수 있음


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,

    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
    "question": "Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt = example_prompt,
    max_length = 300,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country = "Brazil")


'Human: Italy?\nAI\n    Here is what I know:\n    Capital : Rome\n    Language: Italian\n    Good: Pizza and Pasta\n    Currency: Euro\n\nHuman: Greece?\nAI\n    I know this:\n    Capital: Athens\n    Language: Greek\n    Good:Souvlaki and Feta Cheese\n    Currency: Euro\n\nHuman: France?\nAII know this:\n    Capital : Paris\n    Language: French\n    Food: Wine and Cheese\n    Currency: Euro\n\nHuman: What do you know about Brazil?'

## BaseExampleSelector

In [ ]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
# 예제들을 형식화함, 예제의 양이 얼마나 되는지 확인할 수 있음
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
{
    "question": "Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country = "Brazil")


## Serialization and Composition
#### 여러 개의 prompt를 만들어서 병합시키는 작업이 필요함.

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
# prompt 파일을 json 또는 yaml 으로 만들어서 import 시킴

prompt = load_prompt("./prompt.yaml")

prompt.format(country = "korea")

'what is the capital of korea'

In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate
# pipelineprompt는 많은 prompt를 하나로 합쳐줌

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)
intro = PromptTemplate.from_template(
    """You are a role playing assistant.
    And you are impersonating a {character}"""
)

example = PromptTemplate.from_template(
    """This is an example of how you talk:
    
    Human: {example_question}
     You: {example_answer} """
)

start = PromptTemplate.from_template(
    """Start now!
    
    Human: {question}
    You: """
)

# final 프롬프트에 위에서 만들어 놓은 프롬프트를 모두 넣음
final = PromptTemplate.from_template(
    """
{intro} 

{example}

{start}"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

# final 프롬프트를 return
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer = "Arrrrg! That is a secret!! Arg arg!!",
    question="What is your favorite food?"
)

chain = full_prompt | chat
chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrg! That is a secret!! Arg arg!!",
    "question":"what is your favorite food?"

})

Arrr, me favorite food be a hearty plate o' salted fish and hardtack! It be keepin' me belly full on the high seas, matey!

AIMessageChunk(content="Arrr, me favorite food be a hearty plate o' salted fish and hardtack! It be keepin' me belly full on the high seas, matey!")

## Caching
##### 캐싱을 사용하면 LM의 응답을 저장할 수 있음.
##### 답변을 저장하여 재사용하면 답변 시간을 줄일 수 있음.

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler()
    # ]
)

chat.predict("How do you make Italian pasta")

# 첫 번째 답변은 19.5초 소요됨

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller portions for easier handling.\n7. Tak

In [ ]:
chat.predict("How do you make Italian pasta")
# 두 번째 답변은 17.5초 소요됨

In [45]:
chat.predict("How do you make Italian pasta")
# 바로 답변함

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller portions for easier handling.\n7. Tak

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler()
    # ]
)

chat.predict("How do you make Italian pasta")




[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [14.16s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water to enhance the flavor of the pasta.\n\n2. Add pasta: Carefully add the pasta to the boiling water. Use about 100 grams of pasta per person, or adjust according to your preference.\n\n3. Cook pasta: Stir the pasta occasionally to prevent sticking. Cook it according to the package instructions or until it reaches your desired level of doneness. Italians generally prefer their pasta al dente, which means it should be cooked but still firm to the bite.\n\n4. Prepare sauce: While the pasta is cooking, you can prepare the sauce. There are various Italian pasta sauces, such as tomato-based sauces like marinara o

"To make Italian pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water to enhance the flavor of the pasta.\n\n2. Add pasta: Carefully add the pasta to the boiling water. Use about 100 grams of pasta per person, or adjust according to your preference.\n\n3. Cook pasta: Stir the pasta occasionally to prevent sticking. Cook it according to the package instructions or until it reaches your desired level of doneness. Italians generally prefer their pasta al dente, which means it should be cooked but still firm to the bite.\n\n4. Prepare sauce: While the pasta is cooking, you can prepare the sauce. There are various Italian pasta sauces, such as tomato-based sauces like marinara or Bolognese, cream-based sauces like Alfredo, or olive oil-based sauces like aglio e olio. Choose a sauce that complements your taste.\n\n5. Drain pasta: Once the pasta is cooked, carefully drain it in a colander. Do not rinse the pasta, as the

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
set_debug(True)

# db를 지정할 수 있음, 자동으로 cache.db 파일을 만들어줌, 다양한 캐쉬를 만들 수 있음.
set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler()
    # ]
)

chat.predict("How do you make Italian pasta")




[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [17.97s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too thick to mix with a fork, use your hands to knead it. If the dough is too dry, add a little water, one tablespoon at a time, until it comes together. If it's too sticky, add a little more flour.\n5. Knead the dough fo

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too thick to mix with a fork, use your hands to knead it. If the dough is too dry, add a little water, one tablespoon at a time, until it comes together. If it's too sticky, add a little more flour.\n5. Knead the dough for about 5-10 minutes until it becomes smooth and elastic.\n6. Shape the dough into a ball and cover it with a clean kitchen towel or plastic wrap. Let it rest for at least 30 minutes to allow the gluten to relax.\n7. After resting, divide the d

## MEMORY
##### 챗봇에 MEMORY를 추가하지 않으면 챗봇은 아무것도 기억하지 못한다! 
##### OPEN AI는 답변만 하고 답변을 저장하지 않음.

In [6]:
# Conversation Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
# history로 대화 내용이 저장됨
memory.load_memory_variables({})


# 챗모델이 사용할 수 있는 형태로 저장하고 싶을 때
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
# history로 대화 내용이 저장됨
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [11]:
# Conversation Buffer Window Memory

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

memory.load_memory_variables({})


{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [12]:
add_message(5,5)

memory.load_memory_variables({})

# memory를 4개 까지 저장하기 때문에 5번째 메시지를 넣은 순간 가장 이전의 메시지인 '1'이 사라지게 됨.

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [15]:
# Conversation Summary Memory
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 이 메모리를 사용하면 비용이 발생함
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input" : input}, {"output" : output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is a cool")

In [16]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [18]:
get_history()

# 위의 대화 내용을 요약하면서 저장함.

{'history': 'Nicolas introduces himself as living in South Korea. The AI responds positively to this information, expressing a desire to visit the country because it is so pretty.'}

In [23]:
# Conversation Summary Memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# max_token_limit : 메시지들이 요약되기 전, 메시지 토큰 수의 최대값
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
    )

def add_message(input, output):
    memory.save_context({"input" : input}, {"output" : output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is a cool")

In [24]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is a cool')]}

In [25]:
add_message("South Korea is so pretty", "I wish I could go!!")

In [26]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is a cool'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [27]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!!")

In [28]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is a cool'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!!")]}

In [35]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!!")

In [36]:
get_history()

{'history': [SystemMessage(content='Nicolas introduces himself as living in South Korea. The AI responds positively to this information.'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!!")]}